In [1]:
import pandas as pd
import os
from charset_normalizer import detect
from io import StringIO

# File path
file_path = r'C:\Users\COMPLAB\Downloads\Report_20250203T091632.csv'

# Function to detect file encoding
def detect_encoding(file_path):
    try:
        with open(file_path, 'rb') as f:
            result = detect(f.read())
            return result['encoding'] or 'utf-8'
    except Exception as e:
        print(f"Error detecting encoding: {e}")
        return 'utf-8'

# Function to detect delimiter
def detect_delimiter(file_path, encoding):
    try:
        with open(file_path, 'r', encoding=encoding) as f:
            first_line = f.readline()
            delimiters = [',', '\t', ';', '|']
            max_cols = 0
            best_delimiter = ','
            for delimiter in delimiters:
                f.seek(0)
                try:
                    df_temp = pd.read_csv(StringIO(f.read()), sep=delimiter, nrows=5)
                    if len(df_temp.columns) > max_cols:
                        max_cols = len(df_temp.columns)
                        best_delimiter = delimiter
                except:
                    continue
            return best_delimiter
    except Exception as e:
        print(f"Error detecting delimiter: {e}")
        return '\t'  # Fallback to tab based on file snippet

# Verify file exists
if not os.path.exists(file_path):
    print(f"Error: File '{file_path}' does not exist. Please check the file path.")
    exit()

# Detect encoding and delimiter
encoding = detect_encoding(file_path)
print(f"Detected encoding: {encoding}")
delimiter = detect_delimiter(file_path, encoding)
print(f"Detected delimiter: {delimiter}")

# Read the CSV file
try:
    # Read file content with detected encoding
    with open(file_path, 'r', encoding=encoding) as f:
        file_content = f.read()
    # Load into DataFrame using StringIO
    df = pd.read_csv(StringIO(file_content), sep=delimiter)
except Exception as e:
    print(f"Error reading CSV with detected encoding: {e}")
    try:
        # Fallback to latin1 encoding
        with open(file_path, 'r', encoding='latin1') as f:
            file_content = f.read()
        df = pd.read_csv(StringIO(file_content), sep=delimiter)
    except Exception as e:
        print(f"Error reading CSV with latin1 fallback: {e}")
        print("Could not read the file. Please check the file format, encoding, or content.")
        exit()

# Verify DataFrame
if not isinstance(df, pd.DataFrame):
    print("Error: 'df' is not a pandas DataFrame. Please check the file content.")
    exit()

# Print first few lines for inspection
print("\nFirst few lines of the file:")
try:
    with open(file_path, 'r', encoding=encoding) as f:
        for i, line in enumerate(f):
            if i < 5:
                print(line.strip())
            else:
                break
except Exception as e:
    print(f"Error reading file content: {e}")

# Print column names to verify correct loading
print("\nColumn names in the CSV file:", df.columns.tolist())

# Clean column names (remove leading/trailing spaces, special characters)
try:
    df.columns = df.columns.str.strip().str.replace(r'[^\w\s]', '', regex=True)
except AttributeError:
    print("Error: Unable to process column names. 'df.columns' is not a valid attribute.")
    exit()

# Check if 'Date' and 'User' columns exist (case-insensitive)
required_columns = ['Date', 'User']
found_columns = {col: any(col.lower() in df_col.lower() for df_col in df.columns) for col in required_columns}

if not all(found_columns.values()):
    print("Error: Required columns not found in the CSV file.")
    for col, found in found_columns.items():
        if not found:
            print(f" - Missing column: '{col}'")
            similar_cols = [df_col for df_col in df.columns if col.lower() in df_col.lower()]
            if similar_cols:
                print(f"   Possible matches: {similar_cols}")
    print("Please check the file and ensure it has columns named 'Date' and 'User' (case-insensitive).")
    exit()

# Proceed with processing
print("File loaded successfully. Proceeding with attendance processing...")
# Add your further processing here

Detected encoding: UTF-8-SIG
Detected delimiter: ,

First few lines of the file:
Date,User,User Group
1/31/2025 19:09,027324(ODIPO BEATRICE AKOTH),1A
1/31/2025 19:01,019424(ELVIS TITANY),1A
1/31/2025 18:15,027824(JOAN JEPKOSGEI),1A
1/31/2025 18:15,049824(MAUREEN AKINYI ONDIEK),1A

Column names in the CSV file: ['Date', 'User', 'User Group']
File loaded successfully. Proceeding with attendance processing...


In [2]:
def extract_name(user_string):
    # Extract name from format like "DSNEHI056024(Monicah Lilian Nyaboke)"
    if '(' in user_string and ')' in user_string:
        return user_string.split('(')[1].split(')')[0]
    return user_string

# Extract student names
df['Student Name'] = df['User'].apply(extract_name)

# Convert date string to datetime object
# Adjust the format if your date is in a different format
# Convert date string to datetime object, handling multiple formats
df['Datetime'] = pd.to_datetime(df['Date'], format='mixed', errors='coerce', dayfirst=False)
# Check for unparsed dates
if df['Datetime'].isna().any():
    print("\nWarning: Some dates could not be parsed. Unparsed dates:")
    print(df[df['Datetime'].isna()][['Date', 'User']])
    # Optionally, save problematic rows for inspection
    df[df['Datetime'].isna()].to_csv('unparsed_dates.csv', index=False)
# Extract date, time, and hour components
df['Date'] = df['Datetime'].dt.date
df['Time'] = df['Datetime'].dt.time
df['Hour'] = df['Datetime'].dt.hour


# Define session periods
def get_session(hour):
    if 6 <= hour < 10:
        return 'Morning'
    elif 11 <= hour < 13:
        return 'Afternoon'
    elif 14 <= hour < 16:
        return 'Evening'
    else:
        return 'Outside Sessions'

df['Session'] = df['Hour'].apply(get_session)

# Create a pivot table to track attendance by student, day, and session
# First, create a unique record for each student-date-session combination
attendance_records = df.drop_duplicates(subset=['Student Name', 'Date', 'Session'])

# Filter out records outside our defined sessions
attendance_records = attendance_records[attendance_records['Session'] != 'Outside Sessions']

# Create attendance matrix
pivot_df = pd.pivot_table(
    attendance_records,
    values='User',  # Any column would work here
    index=['Student Name', 'Date','User Group'],
    columns=['Session'],
    aggfunc='count',  # Count occurrences
    fill_value=0
)

# Reset index to convert back to regular DataFrame
attendance_matrix = pivot_df.reset_index()

# Replace counts with "Yes" and missing values with "No"
for session in ['Morning', 'Afternoon', 'Evening']:
    if session in attendance_matrix.columns:
        attendance_matrix[session] = attendance_matrix[session].apply(lambda x: "Yes" if x > 0 else "No")
    else:
        attendance_matrix[session] = "No"

# Convert Date to string format for better readability
attendance_matrix['Date'] = attendance_matrix['Date'].astype(str)

# Export to CSV

attendance_matrix.to_csv(r'C:\Users\COMPLAB\Desktop\AttendanceOutput\file9.csv')

 

# Optional: Show the first few rows of the result
print("\nFirst few rows of attendance report:")
print(attendance_matrix.head())

# Summary statistics
print("\nSummary statistics:")
total_students = attendance_matrix['Student Name'].nunique()
total_days = attendance_matrix['Date'].nunique()

print(f"Total unique students: {total_students}")
print(f"Total days analyzed: {total_days}")

# Calculate attendance rates per session
for session in ['Morning', 'Afternoon', 'Evening']:
    if session in attendance_matrix.columns:
        attendance_rate = (attendance_matrix[session] == 'Yes').mean() * 100
        print(f"{session} session attendance rate: {attendance_rate:.2f}%")


First few rows of attendance report:
Session    Student Name        Date User Group Afternoon Evening Morning
0              20959031  2025-01-29         1E        No      No     Yes
1              20959031  2025-01-31         1E        No      No     Yes
2        ACHIENG YVONNE  2025-01-27     Autism       Yes     Yes     Yes
3        ACHIENG YVONNE  2025-01-28     Autism       Yes      No     Yes
4        ACHIENG YVONNE  2025-01-29     Autism        No      No     Yes

Summary statistics:
Total unique students: 465
Total days analyzed: 5
Morning session attendance rate: 95.43%
Afternoon session attendance rate: 49.02%
Evening session attendance rate: 47.78%
